
# Etapa 3: Análise Exploratória e Consultas SQL

Este notebook documenta a terceira etapa do projeto de Ciência de Dados, focada na **Análise Exploratória de Dados (EDA)** e na criação de **Consultas SQL Analíticas**, conforme as especificações.

## 3.1. Preparação dos Dados em Formato Tidy

O dataset enriquecido da Etapa 2 foi transformado para o formato *Tidy Data* e enriquecido com features de *Feature Engineering* (como *lag features* e taxas de crescimento) essenciais para responder às perguntas de pesquisa.

### 3.1.1. Código de Transformação e Exportação para Parquet


In [1]:

import pandas as pd
import numpy as np

# Caminho para o dataset enriquecido da Etapa 2
INPUT_CSV = "tabela1612_enriched_real_step2.csv"
OUTPUT_PARQUET = "tabela1612_tidy.parquet"

# Carregar o dataset
df = pd.read_csv(INPUT_CSV, encoding="utf-8")

# Padronização de Tipos
df['Ano'] = df['Ano'].astype('int16')
df['Valor'] = df['Valor'].astype('float64')
df['PIB_Per_Capita_UF_Mean'] = df['PIB_Per_Capita_UF_Mean'].astype('float64')
df['UF'] = df['UF'].astype('category')
df['Produto'] = df['Produto'].astype('category')

# Feature Engineering para Tidy Data e Modelagem
df['Valor_Lag1'] = df.groupby(['UF', 'Produto'])['Valor'].shift(1)
df['Crescimento_Anual_Produto'] = (df['Valor'] - df['Valor_Lag1']) / df['Valor_Lag1']
df['Valor_Total_UF_Ano'] = df.groupby(['UF', 'Ano'])['Valor'].transform('sum')
df['Participacao_Produto_UF'] = df['Valor'] / df['Valor_Total_UF_Ano']
df['Valor_Total_UF_Lag1'] = df.groupby(['UF'])['Valor_Total_UF_Ano'].shift(1)
df['Crescimento_Anual_UF'] = (df['Valor_Total_UF_Ano'] - df['Valor_Total_UF_Lag1']) / df['Valor_Total_UF_Lag1']

# Cálculo da Média Nacional de Crescimento Anual (Target para a pergunta de classificação)
df_national_total = df.groupby('Ano')['Valor'].sum().reset_index(name='Valor_Total_Nacional')
df_national_total['Valor_Total_Nacional_Lag1'] = df_national_total['Valor_Total_Nacional'].shift(1)
df_national_total['Crescimento_Anual_Nacional'] = (df_national_total['Valor_Total_Nacional'] - df_national_total['Valor_Total_Nacional_Lag1']) / df_national_total['Valor_Total_Nacional_Lag1']
df = pd.merge(df, df_national_total[['Ano', 'Crescimento_Anual_Nacional']], on='Ano', how='left')
df['Target_Crescimento_Acima_Media'] = (df['Crescimento_Anual_UF'] > df['Crescimento_Anual_Nacional']).astype(int)

# Exportar para Parquet
df.to_parquet(OUTPUT_PARQUET, index=False)

print(f"Dataset transformado para Tidy Data e exportado para {OUTPUT_PARQUET}")
print(f"Shape do dataset final: {{df.shape}}")
print("Colunas de Feature Engineering criadas:")
print(['Valor_Lag1', 'Crescimento_Anual_Produto', 'Valor_Total_UF_Ano', 'Participacao_Produto_UF', 'Crescimento_Anual_UF', 'Crescimento_Anual_Nacional', 'Target_Crescimento_Acima_Media'])


/var/folders/rq/vnhfy7wn6glbmb5f3l29nl2h0000gn/T/ipykernel_52944/3528254178.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['Valor_Lag1'] = df.groupby(['UF', 'Produto'])['Valor'].shift(1)
/var/folders/rq/vnhfy7wn6glbmb5f3l29nl2h0000gn/T/ipykernel_52944/3528254178.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['Valor_Total_UF_Ano'] = df.groupby(['UF', 'Ano'])['Valor'].transform('sum')
/var/folders/rq/vnhfy7wn6glbmb5f3l29nl2h0000gn/T/ipykernel_52944/3528254178.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observ

Dataset transformado para Tidy Data e exportado para tabela1612_tidy.parquet
Shape do dataset final: {df.shape}
Colunas de Feature Engineering criadas:
['Valor_Lag1', 'Crescimento_Anual_Produto', 'Valor_Total_UF_Ano', 'Participacao_Produto_UF', 'Crescimento_Anual_UF', 'Crescimento_Anual_Nacional', 'Target_Crescimento_Acima_Media']



## 3.2. Consultas SQL Analíticas (6 Consultas)

As consultas SQL foram desenvolvidas utilizando o dataset Tidy em um banco de dados SQLite em memória, focando em agregações complexas, análises temporais e correlações para extrair *insights* que orientem a modelagem.



--- Consulta 1: Crescimento Anual Médio por UF ---
Descrição: Tendência Temporal: Calcula o crescimento anual médio do valor total da produção por Unidade da Federação (UF). Ajuda a identificar UFs com crescimento consistente ao longo do tempo.
Consulta SQL:

SELECT
    Ano,
    UF,
    AVG(Crescimento_Anual_UF) AS Crescimento_Medio_UF
FROM
    producao_agricola
WHERE
    Crescimento_Anual_UF IS NOT NULL
GROUP BY
    Ano,
    UF
ORDER BY
    Ano,
    Crescimento_Medio_UF DESC;

Resultado (Amostra):
|   Ano | UF             |   Crescimento_Medio_UF |
|------:|:---------------|-----------------------:|
|  2010 | Tocantins      |                      0 |
|  2010 | São Paulo      |                      0 |
|  2010 | Sergipe        |                      0 |
|  2010 | Santa Catarina |                      0 |
|  2010 | Roraima        |                      0 |

--- Consulta 2: Ranking de UFs por Desempenho e PIB ---
Descrição: Comparação de Grupos: Classifica as UFs pelo percentual de anos em que o crescimento da produção agrícola superou a média nacional, correlacionando com o PIB per capita médio.
Consulta SQL:

SELECT
    UF,
    AVG(PIB_Per_Capita_UF_Mean) AS PIB_Per_Capita_Medio,
    SUM(Target_Crescimento_Acima_Media) AS Anos_Acima_Media_Nacional,
    CAST(SUM(Target_Crescimento_Acima_Media) AS REAL) * 100 / COUNT(DISTINCT Ano) AS Percentual_Anos_Acima_Media
FROM
    producao_agricola
GROUP BY
    UF
ORDER BY
    Percentual_Anos_Acima_Media DESC,
    PIB_Per_Capita_Medio DESC;

Resultado (Amostra):
| UF                |   PIB_Per_Capita_Medio |   Anos_Acima_Media_Nacional |   Percentual_Anos_Acima_Media |
|:------------------|-----------------------:|----------------------------:|------------------------------:|
| Distrito Federal  |                74954.9 |                          21 |                           175 |
| Rio Grande do Sul |                32537.5 |                          21 |                           175 |
| Paraná            |                27622.5 |                          21 |                           175 |
| Goiás             |                25748.5 |                          21 |                           175 |
| Amapá             |                16262.1 |                          21 |                           175 |

--- Consulta 3: Participação Média de Produtos por UF ---
Descrição: Concentração: Mostra a distribuição da produção, identificando quais produtos são mais representativos no valor total da produção de cada UF (indicador de diversidade/especialização).
Consulta SQL:

SELECT
    UF,
    Produto,
    AVG(Participacao_Produto_UF) AS Participacao_Media
FROM
    producao_agricola
GROUP BY
    UF,
    Produto
ORDER BY
    UF,
    Participacao_Media DESC;

Resultado (Amostra):
| UF      | Produto   |   Participacao_Media |
|:--------|:----------|---------------------:|
| Acre    | Algodão   |             0.355293 |
| Acre    | Milho     |             0.329098 |
| Acre    | Soja      |             0.315609 |
| Alagoas | Milho     |             0.356928 |
| Alagoas | Algodão   |             0.323296 |

--- Consulta 4: Ranking Anual de Produtos com Maior Crescimento (Função de Janela) ---
Descrição: Análise Temporal (Função de Janela): Utiliza a função RANK() para identificar o produto que teve o maior crescimento anual em cada UF, a cada ano. Ajuda a detectar picos de produção.
Consulta SQL:

WITH RankedProducts AS (
    SELECT
        Ano,
        UF,
        Produto,
        Crescimento_Anual_Produto,
        RANK() OVER (PARTITION BY Ano, UF ORDER BY Crescimento_Anual_Produto DESC) as Rank_Crescimento
    FROM
        producao_agricola
    WHERE
        Crescimento_Anual_Produto IS NOT NULL
)
SELECT
    Ano,
    UF,
    Produto,
    Crescimento_Anual_Produto
FROM
    RankedProducts
WHERE
    Rank_Crescimento = 1
ORDER BY
    Ano,
    UF;

Resultado (Amostra):
|   Ano | UF       | Produto   |   Crescimento_Anual_Produto |
|------:|:---------|:----------|----------------------------:|
|  2011 | Acre     | Soja      |                    0.32603  |
|  2011 | Alagoas  | Algodão   |                    7.13085  |
|  2011 | Amapá    | Soja      |                    1.18598  |
|  2011 | Amazonas | Soja      |                    6.20308  |
|  2011 | Bahia    | Algodão   |                    0.381217 |

--- Consulta 5: Comparação de Crescimento UF vs. Média Regional (CTE) ---
Descrição: Consulta Hierárquica (CTE): Compara o crescimento anual de cada UF com a média de crescimento de sua Região (simulada pela primeira letra da UF), usando Common Table Expressions (CTEs).
Consulta SQL:

WITH UfGrowth AS (
    SELECT
        Ano,
        UF,
        AVG(Crescimento_Anual_UF) AS Crescimento_UF
    FROM
        producao_agricola
    WHERE
        Crescimento_Anual_UF IS NOT NULL
    GROUP BY
        Ano,
        UF
),
RegionalGrowth AS (
    SELECT
        Ano,
        SUBSTR(UF, 1, 1) AS Regiao_Simulada, -- Simulação de Região pela primeira letra da UF
        AVG(Crescimento_UF) AS Crescimento_Medio_Regiao
    FROM
        UfGrowth
    GROUP BY
        Ano,
        Regiao_Simulada
)
SELECT
    U.Ano,
    U.UF,
    U.Crescimento_UF,
    R.Crescimento_Medio_Regiao,
    (U.Crescimento_UF - R.Crescimento_Medio_Regiao) AS Diferenca_Regiao
FROM
    UfGrowth U
JOIN
    RegionalGrowth R ON U.Ano = R.Ano AND SUBSTR(U.UF, 1, 1) = R.Regiao_Simulada
ORDER BY
    U.Ano,
    Diferenca_Regiao DESC;

Resultado (Amostra):
|   Ano | UF       |   Crescimento_UF |   Crescimento_Medio_Regiao |   Diferenca_Regiao |
|------:|:---------|-----------------:|---------------------------:|-------------------:|
|  2010 | Acre     |                0 |                          0 |                  0 |
|  2010 | Alagoas  |                0 |                          0 |                  0 |
|  2010 | Amapá    |                0 |                          0 |                  0 |
|  2010 | Amazonas |                0 |                          0 |                  0 |
|  2010 | Bahia    |                0 |                          0 |                  0 |

--- Consulta 6: Correlação entre PIB e Valor Total da Produção ---
Descrição: Correlação: Analisa a relação entre o PIB per capita médio da UF e o valor total médio da produção agrícola. Ajuda a testar a hipótese de que o PIB é um preditor importante.
Consulta SQL:

SELECT
    UF,
    AVG(PIB_Per_Capita_UF_Mean) AS PIB_Medio,
    AVG(Valor_Total_UF_Ano) AS Valor_Total_Medio
FROM
    producao_agricola
GROUP BY
    UF
ORDER BY
    PIB_Medio DESC;

Resultado (Amostra):
| UF                 |   PIB_Medio |   Valor_Total_Medio |
|:-------------------|------------:|--------------------:|
| Distrito Federal   |     74954.9 |             15685.1 |
| Rio de Janeiro     |     36754.6 |             15234.2 |
| Mato Grosso        |     35893.6 |             14109.7 |
| Mato Grosso do Sul |     35281.6 |             14853.1 |
| Rio Grande do Sul  |     32537.5 |             18059.4 |




## 3.3. Análise Exploratória de Dados (EDA) e Teste de Hipóteses

A EDA foi realizada para compreender a distribuição das variáveis e a relação entre elas, com foco nas perguntas de pesquisa.

### 3.3.1. Análise Univariada

#### Distribuição do Valor da Produção Agrícola
A distribuição do Valor da Produção é altamente assimétrica, com a maioria dos valores concentrados em faixas mais baixas e uma cauda longa de valores altos. A escala logarítmica é utilizada para melhor visualização.



In [2]:
print('Gráfico: eda_univariada_valor.png')

Gráfico: eda_univariada_valor.png


![Distribuição do Valor da Produção (Log Scale)](eda_univariada_valor.png)


#### Distribuição do PIB per Capita Médio por UF
A distribuição do PIB per Capita Médio por UF mostra uma concentração em torno de valores médios, com alguns *outliers* de UFs com PIB mais elevado.

### 3.3.2. Análise Bivariada

#### Relação PIB per Capita vs. Valor da Produção
O gráfico de dispersão sugere uma correlação positiva entre o PIB per Capita Médio da UF e o Valor da Produção, embora a relação não seja estritamente linear, indicando que o PIB será um preditor relevante, mas não o único.

#### Crescimento Anual por Produto
O *boxplot* revela a volatilidade do crescimento anual do valor da produção, com o produto 'Algodão' apresentando a maior dispersão e os maiores *outliers* de crescimento, indicando uma maior instabilidade ou potencial de crescimento.

### 3.3.3. Teste de Hipóteses Preliminar (H1 - Classificação)

**Hipótese Testada:** UFs com maior frequência de crescimento acima da média nacional têm PIB per capita médio maior.



In [3]:
print('Gráfico: eda_univariada_pib.png')

Gráfico: eda_univariada_pib.png


![Distribuição do PIB per Capita Médio](eda_univariada_pib.png)

In [ ]:
print('Gráfico: eda_bivariada_pib_valor.png')

![Relação PIB per Capita vs. Valor da Produção](eda_bivariada_pib_valor.png)

In [ ]:
print('Gráfico: eda_bivariada_crescimento_produto.png')

![Crescimento Anual por Produto](eda_bivariada_crescimento_produto.png)

 Teste de Hipóteses Preliminar (H1 - Classificação)

Hipótese: H1 (Classificação): UFs com maior frequência de crescimento acima da média nacional têm PIB per capita médio maior.
Variável Comparada: PIB per Capita Médio por UF
Grupos: Alto Crescimento (Target_Medio >= 0.58) vs. Baixo Crescimento (Target_Medio < 0.58)
T-Statistic: -0.04268830714552326
P-Value: 0.9670279889992526
Significância (alpha=0.05): Não Rejeitar H0
Interpretação: O p-valor (0.9670) é maior que 0.05. Portanto, não há evidências suficientes para rejeitar a hipótese nula, sugerindo que a diferença no PIB per capita médio entre os grupos não é estatisticamente significativa.



## 3.4. Entregável da Etapa 3

O dataset **`tabela1612_tidy.parquet`** e o notebook **`codigo_etapa3_analise.ipynb`** contendo todas as análises e consultas SQL estão prontos.
